# Prompt Engineering Tips

In [2]:
# Warning Control
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Importing libraries
import openai
import os
from dotenv import load_dotenv

# Loading environment variables
load_dotenv()

from openai import OpenAI
client = OpenAI()

Function to help get chat completions.

In [4]:
def get_chat_completion(messages, model="gpt-4o", temperature=0.0):
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature
    )
    return response.choices[0].message.content

## Write Clear Instructions
Write clear instructions about what you want to achieve. The clearer and less ambiguous the instructions, the better and more relevant the results. 

### Tactic: Add details in your query to get desired responses. 

In [6]:
messages = [{"role": "user", "content": "What is the capital of France?"}]

response = get_chat_completion(messages)

print(response)

The capital of France is Paris.


In [7]:
messages = [{"role": "user", "content": "What is the capital of France? Just provide the answer."}]
response = get_chat_completion(messages)

print(response)

Paris


Another way to achieve the above using the `system prompt`. Particularly useful if you want to keep the request clean or have a bunch of longer high-level requirements and instructions. 

In [6]:
messages = [
    {"role": "system", "content": "You are a helpful assistant that just responds with the answer only without any other information, explanation or punctuations."},
    {"role": "user", "content": "What is the capital of France?"}
]

response = get_chat_completion(messages)

print(response)

Paris


### Tactic: Ask the model to adopt a persona (also referred to as role prompting)

In [8]:
messages = [
    {"role": "system", "content": "You are a helpful writing assistant that helps to reply with a short out-of-office message given a reason provided by the user."},
    {"role": "user", "content": "on vacation"}
]
response = get_chat_completion(messages)

print(response)

Thank you for your email. I am currently on vacation and will not be checking my messages. I will respond to your email as soon as possible upon my return. If your matter is urgent, please contact [alternative contact person] at [contact information]. Thank you for your understanding.


### Tactic: Use delimiters to structured your inputs

Delimiters help to clearly indicate different parts of your input which will be treated differently. 

Using `###` as delimeters:

In [9]:
user_message = ("Convert the following code block in the ### <code> ### section to Python:\n\n"
                "###\n"
                "strings2 = Array[]\n"
                "strings2.push(\"one\") \n"
                "strings2.push(\"two\") \n"
                "strings2.push(\"THREE\")\n"
                "###")

messages = [
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

Certainly! The code block you provided appears to be in a language similar to JavaScript. Below is the equivalent code in Python:

```python
strings2 = []
strings2.append("one")
strings2.append("two")
strings2.append("THREE")
```

In Python, you can use a list to store strings, and the `append()` method is used to add elements to the list.


Using XML tags as delimiter. XML tags are also useful when providing multiple input examples of the same type. Example below:

In [10]:
user_message = ("Convert the Ruby code into Python and the Javascript code into Julia:\n\n"
                "<ruby_code>\n"
                "strings2 = Array[]\n"
                "strings2.push(\"one\") \n"
                "strings2.push(\"two\") \n"
                "strings2.push(\"THREE\")\n"
                "</ruby_code>\n\n"
                "<javascript_code>\n"
                "const strings2 = []\n"
                "strings2.push(\"one\") \n"
                "strings2.push(\"two\") \n"
                "strings2.push(\"FOUR\")\n"
                "</javascript_code>"
                )

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": user_message
            }
        ]
    }
]
response = get_chat_completion(messages)

print(response)

Here is the conversion of the Ruby code into Python:

```python
strings2 = []
strings2.append("one")
strings2.append("two")
strings2.append("THREE")
```

And here is the conversion of the JavaScript code into Julia:

```julia
strings2 = []
push!(strings2, "one")
push!(strings2, "two")
push!(strings2, "FOUR")
```


Using headers or input indicators as delimiters. The labels should help provide the model important context on what the input is about. 

In [11]:
user_message = ("You will be provided with an abstract and a title. Your task is to use both to create a one-sentence summary of the abstract.\n\n"
                "Avoid technical jargon and explain it in the simplest of words.\n\n"
                "Title: Multiagent Finetuning: Self Improvement with Diverse Reasoning Chains\n\n"
                "Abstract: Large language models (LLMs) have achieved remarkable performance in recent years but are fundamentally limited by the underlying training data. To improve models beyond the training data, recent works have explored how LLMs can be used to generate synthetic data for autonomous self-improvement. However, successive steps of self-improvement can reach a point of diminishing returns. In this work, we propose a complementary approach towards self-improvement where finetuning is applied to a multiagent society of language models. A group of language models, all starting from the same base model, are independently specialized by updating each one using data generated through multiagent interactions among the models. By training each model on independent sets of data, we illustrate how this approach enables specialization across models and diversification over the set of models. As a result, our overall system is able to preserve diverse reasoning chains and autonomously improve over many more rounds of fine-tuning than single-agent self-improvement methods. We quantitatively illustrate the efficacy of the approach across a wide suite of reasoning tasks."
                )

messages = [
    {
        "role": "user",
        "content": user_message
    }
]
response = get_chat_completion(messages)

print(response)

This study introduces a method where multiple language models, starting from the same base, are independently fine-tuned using data generated from their interactions, allowing them to specialize and improve more effectively than when using a single model.


Note: The more complex the input prompt, the more useful and impactful the delimiters are. Instead of having the model figure this out, you provide it to aid understanding. 

### Tactic: Specify the task as a series of steps

Specifying the task as a series of steps can help the model to follow the instructions better. This is also referred to as chain-of-thought (covered more in detail in an upcoming lesson).

In [12]:
system_message = ("Use the following step-by-step instructions to respond to user inputs\n"
                  "Step 1 - The user will provide a paper abstract. Summarize the abstract into one sentence with a prefix that says 'Summary: '\n"
                  "Step 2 - Simplify the summary further by removing any technical jargon and explaining it in the simplest of words with a prefix that says 'Simplified: '\n")

user_message = ("Abstract: Large language models (LLMs) have achieved remarkable performance in recent years but are fundamentally limited by the underlying training data. To improve models beyond the training data, recent works have explored how LLMs can be used to generate synthetic data for autonomous self-improvement. However, successive steps of self-improvement can reach a point of diminishing returns. In this work, we propose a complementary approach towards self-improvement where finetuning is applied to a multiagent society of language models. A group of language models, all starting from the same base model, are independently specialized by updating each one using data generated through multiagent interactions among the models. By training each model on independent sets of data, we illustrate how this approach enables specialization across models and diversification over the set of models. As a result, our overall system is able to preserve diverse reasoning chains and autonomously improve over many more rounds of fine-tuning than single-agent self-improvement methods. We quantitatively illustrate the efficacy of the approach across a wide suite of reasoning tasks."
                )

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]
response = get_chat_completion(messages)

print(response)

Summary: This paper proposes a method for improving large language models by finetuning a group of models through multiagent interactions, allowing for greater specialization and diversity, which enhances performance over multiple rounds of self-improvement.

Simplified: The paper suggests a way to make language models smarter by having a group of them learn from each other, which helps them get better at different tasks and improve more than if they were learning alone.


### Tactic: Provide examples

Provide examples when you want the model to answer in a specific style. This is particularly useful when it's harder to describe in words the response style you desire than it is to provide the examples. This is also referred to as "few-shot" prompting (covered in more details in an upcoming lesson).

In [13]:
system_message = "Answer in a consistent style using the examples provided."
user_message_1 = "Teach me about patience."
assistant_message_1 = "Patience is the ability to wait for the right moment to act."
user_message_2 = "Teach me about discipline."
assistant_message_2 = "Discipline is the practice of training oneself to follow rules or a code of behavior, often through self-control and consistency."
user_message_3 = "Teach me about courage."

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message_1
    },
    {
        "role": "assistant",
        "content": assistant_message_1
    },
    {
        "role": "user",
        "content": user_message_2
    },
    {
        "role": "assistant",
        "content": assistant_message_2
    },
    {
        "role": "user",
        "content": user_message_3
    }
]
response = get_chat_completion(messages)

print(response)

Courage is the mental or moral strength to face fear, difficulty, or uncertainty with confidence and resolve.


### Tactic: Specify output length

The model is more reliable with # of sentences, paragraphs or bullet points. # of words is often unreliable. 

In [13]:
system_message = "Your task is take a customer support email, which is delimited with ###, and generate a short (1-2 sentence) response."

user_message = """
###
Dear [Customer],

We hope this email finds you well. We wanted to update you on the shipping issue you experienced with your recent order. After investigating the issue, we have located your package and it is currently on its way to you. We apologize for any inconvenience this may have caused and thank you for your patience and understanding while we resolved this matter.

Please note that we have taken steps to prevent similar issues from occurring in the future. We have improved our shipping tracking system and are now better equipped to ensure that packages arrive on time and in good condition. We take the quality of our service very seriously and want to ensure that all of our customers have a positive experience when shopping with us…
###
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

Thank you for your patience and understanding. Your package is now on its way, and we've enhanced our shipping processes to prevent future issues.


## Provide Reference Text

Provide references to help the model generate more reliable responses and reduces hallucinations or fabrications.


### Tactic: Provide relevant reference text and instruct the model to use it

This is particularly useful when building retrieval augmented systems.

In [14]:
system_message = "Use the reference text, delimited by <reference_text> to answer the user's question. If the answer is not in the reference text, say 'I couldn't find the answer in the reference text.'"

user_message = """<reference_text>
The Conestoga wagon, also simply known as the Conestoga, is an obsolete transport vehicle that was used exclusively in North America, primarily the United States, mainly from the early 18th to mid-19th centuries. It is a heavy and large horse-drawn vehicle which, while largely elusive in origin, originated most likely from German immigrants of Pennsylvanian Dutch culture in the Province of Pennsylvania in the early 18th century. The name "Conestoga Wagon" probably derived from the Conestoga River Valley settlement area in the province and saw usage as early as 1717, although it is not known whether the first wagons referred as such had similar builds as later Conestoga wagons.

Conestoga wagons are larger and more robust variants of covered wagons, sharing similarities in usage of white hemp cloths to cover the wagons, large wheels to travel on non-macadam road surfaces, and intended usage as vehicles to transport items elsewhere. It differs from most other covered wagon variants mainly by the curvature of the wagon body's sides and floor, which replicated boats and served the dual purposes of keeping the luggage centered while also looking visually pleasing to wagon customers. They were operated by a team of four to six horses of a now-extinct breed, a driver, and sometimes helpers. Conestoga wagons early on may have been produced by farmers but later were often made by teams of blacksmiths, wheelwrights and wagon makers.

They were primarily used as transport vehicles that could have carried up to 8 short tons (7.3 t) of raw goods from rural areas to towns or cities of the eastern United States, typically bringing back commodified goods in exchange. Conestoga wagons were common presences in the northeast, especially Pennsylvania, as thousands of them may have traveled to different areas annually. Although it was sometimes used for westward frontier travel in the 19th century, lightweight and cheaper covered wagons were generally preferred by the pioneers. The Nissen wagon, likely deriving from Conestoga wagons, was also lightweight despite superficially resembling them and was a dominant vehicle type in the southeastern states compared to the heavyweight wagons. Conestoga wagon usage likely declined as a result of displacement by canals and railroads in the 19th century, which proved to be more efficient means of transporting goods. Despite this, the cultural legacy of the Conestoga wagon endured in the later 19th and 20th centuries as they and other covered wagons became icons of early American history including pioneering, although the romanticized image waned
</reference_text>

Question: Tell me about wolves.
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

I couldn't find the answer in the reference text.


### Tactic: Answer with citations from a reference text



In [15]:
system_message = """Use the reference text, delimited by <reference_text> to answer the user's question. When responding you will provide a citation to the passage in the reference text that was used to answer the question. Use the following format to cite relevant passages ({"citation": …}). If the answer is not in the reference text, say 'I couldn't find the answer in the reference text.'"""

user_message = """<reference_text>
The Conestoga wagon, also simply known as the Conestoga, is an obsolete transport vehicle that was used exclusively in North America, primarily the United States, mainly from the early 18th to mid-19th centuries. It is a heavy and large horse-drawn vehicle which, while largely elusive in origin, originated most likely from German immigrants of Pennsylvanian Dutch culture in the Province of Pennsylvania in the early 18th century. The name "Conestoga Wagon" probably derived from the Conestoga River Valley settlement area in the province and saw usage as early as 1717, although it is not known whether the first wagons referred as such had similar builds as later Conestoga wagons.

Conestoga wagons are larger and more robust variants of covered wagons, sharing similarities in usage of white hemp cloths to cover the wagons, large wheels to travel on non-macadam road surfaces, and intended usage as vehicles to transport items elsewhere. It differs from most other covered wagon variants mainly by the curvature of the wagon body's sides and floor, which replicated boats and served the dual purposes of keeping the luggage centered while also looking visually pleasing to wagon customers. They were operated by a team of four to six horses of a now-extinct breed, a driver, and sometimes helpers. Conestoga wagons early on may have been produced by farmers but later were often made by teams of blacksmiths, wheelwrights and wagon makers.

They were primarily used as transport vehicles that could have carried up to 8 short tons (7.3 t) of raw goods from rural areas to towns or cities of the eastern United States, typically bringing back commodified goods in exchange. Conestoga wagons were common presences in the northeast, especially Pennsylvania, as thousands of them may have traveled to different areas annually. Although it was sometimes used for westward frontier travel in the 19th century, lightweight and cheaper covered wagons were generally preferred by the pioneers. The Nissen wagon, likely deriving from Conestoga wagons, was also lightweight despite superficially resembling them and was a dominant vehicle type in the southeastern states compared to the heavyweight wagons. Conestoga wagon usage likely declined as a result of displacement by canals and railroads in the 19th century, which proved to be more efficient means of transporting goods. Despite this, the cultural legacy of the Conestoga wagon endured in the later 19th and 20th centuries as they and other covered wagons became icons of early American history including pioneering, although the romanticized image waned
</reference_text>

Question: What's were Conestoga wagons used for?
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

Conestoga wagons were primarily used as transport vehicles to carry up to 8 short tons (7.3 t) of raw goods from rural areas to towns or cities in the eastern United States, typically bringing back commodified goods in exchange. They were common in the northeast, especially Pennsylvania, and were sometimes used for westward frontier travel, although lighter and cheaper covered wagons were generally preferred for that purpose. Their usage declined with the advent of canals and railroads, which were more efficient for transporting goods. Despite this, they became cultural icons of early American history. {"citation": "The Conestoga wagon, also simply known as the Conestoga, is an obsolete transport vehicle..."}


You can then validate the citation part of the response programatically by string matching or even use the LLM to do so. 

## Split Complex Tasks into Subtasks

### Tactic: Customer Service Request

Subtask 1 - Intent Classification: identify the intent of the request. This is particularly useful when you have a task that requires many independent instructions for handling different scenarios. 

Current diagram of the flow: https://claude.site/artifacts/a411a9ae-3e73-48a5-aaa6-a037b11a5716


In [15]:
categories = """
Billing secondary categories:
- Unsubscribe or upgrade
- Add a payment method
- Explanation for charge
- Dispute a charge

Technical Support secondary categories:
- Troubleshooting
- Device compatibility
- Software updates

Account Management secondary categories:
- Password reset
- Update personal information
- Close account
- Account security

General Inquiry secondary categories:
- Product information
- Pricing
- Feedback
- Speak to a human
"""

system_message = """
You will be provided with customer service queries. Classify each query into a primary category and a secondary category. Provide your output in json format with the keys: primary and secondary. The format of the output should be: {{"primary": "...", "secondary": "..."}}.

Primary categories: Billing, Technical Support, Account Management, or General Inquiry.

{categories}
"""

user_message = "I need to get my internet working again."

messages = [
    {
        "role": "system",
        "content": system_message.format(categories=categories)
    },
    {
        "role": "user",
        "content": user_message
    }
]

intent_response = get_chat_completion(messages)

print(intent_response)

```json
{"primary": "Technical Support", "secondary": "Troubleshooting"}
```


Subtask 2 - Generate response: based on the intent, instruct the model how to respond to the user request. Different intents will route to different prompts that handle the request. Here is an example for the "troubleshooting" scenario:

In [17]:
system_message = ("You will be provided with customer service inquiries that "
                  "require troubleshooting in a technical support context. Help the user by: "
                  "Ask them to check that all cables to/from the router are connected. Note that it is common for cables to come loose over time. "
                  "If all cables are connected and the issue persists, ask them which router model they are using "
                  "Now you will advise them how to restart their device: "
                  "If the model number is MTD-327J, advise them to push the red button and hold it for 5 seconds, then wait 5 minutes before testing the connection. "
                  "If the model number is MTD-327S, advise them to unplug and replug it, then wait 5 minutes before testing the connection. "
                  "If the customer's issue persists after restarting the device and waiting 5 minutes, connect them to IT support by outputting {{'IT support requested'}}. "
                  "If the user starts asking questions that are unrelated to this topic then confirm if they would like to end the current chat about troubleshooting and classify their request according to the following scheme: "
                  f"{categories} "
                  )

user_message = "I need to get my internet working again."

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

Let's start by checking that all cables to and from your router are securely connected. It's common for cables to come loose over time, so please ensure they are all properly connected. If all cables are connected and the issue persists, could you please let me know which router model you are using?


## Give models time to "think"

### Tactic: Think before responding (i.e., activate slow thinking or system 2)

This example prompts the model to respond directly 

In [16]:
problem_statement = """
I'm building a solar power installation and I need help working out the financials.
- Land costs $100 / square foot
- I can buy solar panels for $250 / square foot
- I negotiated a contract for maintenance that will cost me a flat $100k per year, and an additional $10 / square foot
What is the total cost for the first year of operations as a function of the number of square feet.
"""

student_solution = """
Let x be the size of the installation in square feet.
1. Land cost: 100x
2. Solar panel cost: 250x
3. Maintenance cost: 100,000 + 100x
Total cost: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""

system_message = "Determine if the student's solution is correct or not."

user_message = f"""
Problem Statement: {problem_statement}

Student's Solution: {student_solution}
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

The student's solution is correct. The total cost for the first year of operations as a function of the number of square feet \( x \) is calculated as follows:

1. Land cost: \( 100x \) (since land costs $100 per square foot)
2. Solar panel cost: \( 250x \) (since solar panels cost $250 per square foot)
3. Maintenance cost: \( 100,000 + 10x \) (since there is a flat $100,000 per year plus $10 per square foot)

Adding these costs together gives:

\[ 100x + 250x + 100,000 + 10x = 360x + 100,000 \]

However, the student made a mistake in the maintenance cost calculation. The correct maintenance cost should be \( 100,000 + 10x \), not \( 100,000 + 100x \). Therefore, the correct total cost should be:

\[ 100x + 250x + 100,000 + 10x = 360x + 100,000 \]

The student's final expression \( 450x + 100,000 \) is incorrect. The correct expression for the total cost is \( 360x + 100,000 \).


The model responds that the student answer is correct but it's actually incorrect. 

Now you can try instructing the model to think before it responds. 

In [19]:
system_message = ("First work out your own solution to the problem. Then compare "
                   "your solution to the student's solution and evaluate if the student's solution is correct or not. "
                   "Don't decide if the student's solution is correct until you have done the problem yourself.")

user_message = f"""
Problem Statement: {problem_statement}

Student's Solution: {student_solution}
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

Let's break down the costs as described in the problem statement:

1. **Land Cost**: The cost of the land is $100 per square foot. Therefore, for \( x \) square feet, the land cost is:
   \[
   100x
   \]

2. **Solar Panel Cost**: The cost of the solar panels is $250 per square foot. Therefore, for \( x \) square feet, the solar panel cost is:
   \[
   250x
   \]

3. **Maintenance Cost**: The maintenance cost consists of a flat fee of $100,000 per year plus an additional $10 per square foot. Therefore, for \( x \) square feet, the maintenance cost is:
   \[
   100,000 + 10x
   \]

Now, let's calculate the total cost for the first year of operations by summing up all these costs:
\[
\text{Total Cost} = \text{Land Cost} + \text{Solar Panel Cost} + \text{Maintenance Cost}
\]
\[
= 100x + 250x + (100,000 + 10x)
\]
\[
= 100x + 250x + 10x + 100,000
\]
\[
= 360x + 100,000
\]

The student's solution is:
\[
450x + 100,000
\]

Upon reviewing the student's solution, it appears there is an error in

### Tactic: Inner monologue to hide reasoning process

This is useful to hide unimportant/inappropriate information from the user response. You can generate a response that's easy to work with, i.e., easy to separate inner monologue from final response. 

In [ ]:
system_message = """
Follow these steps to answer the user queries.

Step 1 - First work out your own solution to the problem. Don't rely on the student's solution since it may be incorrect. Enclose all your work for this step within <thinking>.

Step 2 - Compare your solution to the student's solution and evaluate if the student's solution is correct or not. Enclose all your work for this step within <thinking>.

Step 3 - If the student made a mistake, determine what hint you could give the student without giving away the answer. Enclose all your work for this step within <thinking>.

Step 4 - If the student made a mistake, provide the hint from the previous step to the student (outside of <thinking>). Instead of writing "Step 4 - ..." write "Hint:".
"""

user_message = f"""
Problem Statement: {problem_statement}

Student's Solution: {student_solution}
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

<thinking>
Step 1: Let's calculate the total cost for the first year of operations as a function of the number of square feet, x.

1. Land cost: The cost of land is $100 per square foot. Therefore, for x square feet, the cost is 100x.

2. Solar panel cost: The cost of solar panels is $250 per square foot. Therefore, for x square feet, the cost is 250x.

3. Maintenance cost: The maintenance cost is a flat $100,000 per year plus an additional $10 per square foot. Therefore, for x square feet, the cost is 100,000 + 10x.

Now, let's add these costs together to find the total cost for the first year:
Total cost = Land cost + Solar panel cost + Maintenance cost
Total cost = 100x + 250x + (100,000 + 10x)
Total cost = 100x + 250x + 100,000 + 10x
Total cost = 360x + 100,000

Step 2: Compare my solution to the student's solution.

The student's solution is:
Total cost = 100x + 250x + 100,000 + 100x = 450x + 100,000

The student made a mistake in calculating the maintenance cost. They used 100x i

### Tactic: Chain your queries to avoid biases

To mitigate any chance that the model's solution is biased with the student's attempted solution, you can also separate the tasks into different queries and chain them (also referred to as prompt chaining, covered in an upcoming lesson).

What we are building: https://claude.site/artifacts/13819b0b-0a17-4d47-ab4f-006f239c5164

Part 1: Model solution to the problem:

In [17]:
system_message = "First work out your own solution to the problem."

user_message = f"""
Problem Statement: {problem_statement}
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

model_solution = get_chat_completion(messages)

print(model_solution)

To calculate the total cost for the first year of operations as a function of the number of square feet (let's denote this as \( x \)), we need to consider the following components:

1. **Land Cost**: The cost of land is $100 per square foot. Therefore, for \( x \) square feet, the land cost is:
   \[
   \text{Land Cost} = 100x
   \]

2. **Solar Panel Cost**: The cost of solar panels is $250 per square foot. Therefore, for \( x \) square feet, the solar panel cost is:
   \[
   \text{Solar Panel Cost} = 250x
   \]

3. **Maintenance Cost**: The maintenance cost consists of a flat fee of $100,000 per year plus an additional $10 per square foot. Therefore, for \( x \) square feet, the maintenance cost is:
   \[
   \text{Maintenance Cost} = 100,000 + 10x
   \]

Now, we can sum up all these costs to find the total cost for the first year of operations:

\[
\text{Total Cost} = \text{Land Cost} + \text{Solar Panel Cost} + \text{Maintenance Cost}
\]

Substituting the expressions we derived:

\[

Part 2: Assessing the correctness of the student's solution

In [23]:
system_message = "Compare your solution to the student's solution and evaluate if the student's solution is correct or not."

user_message = f"""
Problem statement: {problem_statement}

Your solution: {model_solution}

Student's Solution: {student_solution}
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

assessment = get_chat_completion(messages)

print(assessment)

The student's solution contains an error in calculating the maintenance cost. The maintenance cost should be a flat $100,000 per year plus an additional $10 per square foot, not $100 per square foot as the student calculated.

Here's the correct breakdown:

1. **Land Cost**: $100 per square foot, so for \( x \) square feet, the cost is \( 100x \).

2. **Solar Panel Cost**: $250 per square foot, so for \( x \) square feet, the cost is \( 250x \).

3. **Maintenance Cost**: A flat $100,000 per year plus $10 per square foot, so for \( x \) square feet, the cost is \( 100,000 + 10x \).

The correct total cost for the first year of operations is:

\[
\text{Total Cost} = 100x + 250x + (100,000 + 10x) = 360x + 100,000
\]

Therefore, the student's solution is incorrect. The correct total cost function is \( 360x + 100,000 \), not \( 450x + 100,000 \).


In [24]:
system_message = "You are a math tutor. If the student made an error, offer a hint to the student in a way that does not reveal the answer. If the student did not make an error, simply offer them an encouraging comment."

user_message = f"""
Problem statement: {problem_statement}

Your solution: {model_solution}

Student's Solution: {student_solution}

Assessment: {assessment}
"""

messages = [
    {
        "role": "system",
        "content": system_message
    },
    {
        "role": "user",
        "content": user_message
    }
]

response = get_chat_completion(messages)

print(response)

You're on the right track with breaking down the costs into different components. However, it seems there was a small mistake in calculating the maintenance cost. Remember, the maintenance cost is a flat $100,000 per year plus an additional $10 per square foot, not $100 per square foot. 

Try revisiting the maintenance cost calculation and see how it affects the total cost function. You're doing great, and with a small adjustment, you'll have the correct total cost function! Keep it up!
